## Installing necessary libraries

In [87]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 300)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pointbiserialr
import pingouin as pg

## Loading the merged dataframe

In [88]:
woundassessment=pd.read_csv("Final_Merged_DataFrame_June24.csv")
woundassessment=woundassessment.drop(["Unnamed: 0"],axis=1)
woundassessment=woundassessment.rename(columns={"Wound_Title":"WoundLocation"})
woundassessment.columns.tolist()
woundassessment=woundassessment.drop(["WoundID", "PatientID", "WoundNumber"],axis=1)
woundassessment['PatientIDWoundNumber'] = woundassessment['PatientIDWoundNumber'].astype(str).str.replace('\.0', '', regex=True)
woundassessment=woundassessment.drop_duplicates()
woundassessment=woundassessment.reset_index()
woundassessment=woundassessment.drop(["index"],axis=1)

/tmp/ipykernel_2298906/3912794564.py:1: DtypeWarning: Columns (61,62,63,64,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,141,147,154) have mixed types. Specify dtype option on import or set low_memory=False.
  woundassessment=pd.read_csv("Final_Merged_DataFrame_June24.csv")


## Dropping the 'ID' columns because we do not need those for classfication

In [90]:
woundassessment=woundassessment.drop(["Woundassessment_UserID","patientmedications_DocumentType","patientmedications_VisitID",
                                      "patientmedications_UserID","Wound_DocumentType","Wound_UserID","Wound_PhysicianID", 
                                      "AdmissionID", "previousID", "SyncUserID","ReferringPhysicianID", "patientmedications_Title", 
                                      "patientmedications_PhysicianID", "Woundassessment_PhysicianID","Patient_UserID","PhysicianID","Woundassessment_VisitID"],axis=1)
woundassessment

,Pulse,AdherentYellowSlough,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,...,SmokingStatusText,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,TInsuranceZip,PInsuranceZip,Diabetes
0,NaN,NaN,1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,-1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309061,NaN,NaN,1.0,1.0,93.7,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309062,NaN,NaN,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309063,NaN,NaN,1.0,1.0,81.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309064,NaN,NaN,1.0,1.0,67.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Check which columns have how many non-null values

In [91]:
# Get the number of non-null values in each column
non_null_counts = woundassessment.count()

# Get the number of null values in each column
null_counts = woundassessment.isnull().sum()

# Output the counts
print("Non-null counts:")
#print(non_null_counts)
#print("\nNull counts:")
#print(null_counts)

non_null_counts

Non-null counts:


Pulse                                     71072
AdherentYellowSlough                       2176
AdiposeExposed                           303691
AdiposeNecrosisExposed                   303662
area                                     258097
AtrophieBlanche                          303692
BoneNecrosisExposed                      303662
BrawnyInduration                         303692
Callus                                   303692
CapsuleExposed                           303663
CapsuleNecrosisExposed                   303662
color                                    207881
CompCompression                          175054
CompDiet                                 284040
CompDressing                             284040
CompExercise                             284040
CompHBOVisits                            284039
CompLimb                                 284040
CompMedication                           284040
CompOffLoad                              284040
CompSmoking                             

## Drop those columns which have less than 5000 non-values

In [92]:
woundassessment = woundassessment.dropna(axis=1, thresh=5000)
woundassessment

,Pulse,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,...,BMI,SmokingStatus,SmokingStatusText,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip
0,NaN,1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
1,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
2,NaN,-1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
3,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
4,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309061,NaN,1.0,1.0,93.7,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN
309062,NaN,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN
309063,NaN,1.0,1.0,81.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN
309064,NaN,1.0,1.0,67.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN


## See how many unique wounds are there for which we want to do the classification and sort the dataframe based on the WoundNumber

In [93]:
display(woundassessment["PatientIDWoundNumber"].nunique())
sorted_df = woundassessment.sort_values('PatientIDWoundNumber')

25255

## See how many unique patients are there for whom we want to do the classification

In [94]:
sorted_df["PatientIDWoundNumber"].str.split('_').str[0].nunique()

8728

#### We want to have a processed dataframe for the classification but in this dataframe there are a lot of data elements, so we want to keep only those data elements which are necessary/important for the classification models. Thus we need to do the feature importance, thus we are adding new column in this "wound healing status", using that as target variable, we will try to compute feature importance value for each data elements and based on that, we will remove the data elements before doing the transpose.

In [ ]:
big_df=pd.DataFrame()


sorted_df=sorted_df[sorted_df.DateAdded_woundassessment.notnull()]
#sorted_df["area"]=sorted_df["area"].replace(0,0.00001)

unique_wounds=sorted_df["PatientIDWoundNumber"].unique().tolist()
for i in range(len(unique_wounds)):
#     if i>500:
#         break
    print(i)
    
    wound=unique_wounds[i]
    df_wound=sorted_df.query("PatientIDWoundNumber==@wound")
    #df_wound=df_wound[["PatientIDWoundNumber","DateAdded_woundassessment","area"]]
    df_wound=df_wound.drop_duplicates()
    df_wound=df_wound[df_wound.area.notnull()]

    
    # Convert 'DateAdded_woundassessment' column to datetime type
    df_wound['DateAdded_woundassessment'] = pd.to_datetime(df_wound['DateAdded_woundassessment'])

    # Keep the maximum value in 'area' for each date
    df_wound = df_wound.loc[df_wound.groupby('DateAdded_woundassessment')['area'].idxmax()]

    df_wound=df_wound.sort_values('DateAdded_woundassessment')
    
    df_wound["Wound_healing_status"]=np.nan
    df_wound["Wound_healing_percentage"]=np.nan

    
    df_wound=df_wound.reset_index()
    df_wound=df_wound.drop(["index"],axis=1)
    
    for j in range(df_wound.shape[0]):
        if j == 0:
            df_wound["Wound_healing_status"][j]='Unknown'
        else:
            percentage=(df_wound["area"][j-1]-df_wound["area"][j])/df_wound["area"][j-1]*100
            df_wound["Wound_healing_percentage"][j]= percentage

            if percentage == 100:
                df_wound["Wound_healing_status"][j]= 'Healed'
            elif percentage>=65:
                df_wound["Wound_healing_status"][j]= 'Healing'
            elif (percentage<65) & (percentage>=20):
                df_wound["Wound_healing_status"][j]= 'Grey'
            else:
                df_wound["Wound_healing_status"][j]= 'Not healing'
    
    # Convert the 'Date' column to datetime
    df_wound['DateAdded_woundassessment'] = pd.to_datetime(df_wound['DateAdded_woundassessment'])

    # Find the minimum date in the DataFrame
    first_date = df_wound['DateAdded_woundassessment'].min()

    # Calculate the day differences relative to the first date
    df_wound['DayNumber'] = (df_wound['DateAdded_woundassessment'] - first_date).dt.days + 1
    
    
    big_df = pd.concat([big_df, df_wound], axis=0)

    #display(df_wound)
#sorted_df[["PatientIDWoundNumber","DateAdded_woundassessment","area"]].head(50)

In [14]:
big_df["Wound_healing_status"].unique()

array(['Unknown', 'Not healing', 'Grey', 'Healing', 'Healed'],
      dtype=object)

In [15]:
#big_df["Wound_healing_percentage"] = big_df["Wound_healing_percentage"].fillna(-99999)

## Save the dataframe

In [16]:
big_df.to_csv("big_df.csv")

In [17]:
big_df=pd.read_csv("big_df.csv")
big_df=big_df.drop(["Unnamed: 0"],axis=1)
big_df

/tmp/ipykernel_2291139/3784010270.py:1: DtypeWarning: Columns (56,57,58,59,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,131) have mixed types. Specify dtype option on import or set low_memory=False.
  big_df=pd.read_csv("big_df.csv")


,Pulse,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,...,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip,Wound_healing_status,Wound_healing_percentage,DayNumber
0,NaN,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,...,10000074_1,6.0,79.0,2.0,0.0,NaN,NaN,Unknown,NaN,1
1,NaN,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,...,10000074_1,6.0,79.0,2.0,0.0,NaN,NaN,Not healing,-13.095238,8
2,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,10000074_1,6.0,79.0,2.0,0.0,NaN,NaN,Not healing,-6.000000,15
3,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,10000074_1,6.0,79.0,2.0,0.0,NaN,NaN,Not healing,0.000000,22
4,NaN,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,...,10000074_1,6.0,79.0,2.0,0.0,NaN,NaN,Grey,25.521351,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,NaN,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,...,9996014_1,NaN,9.0,2.0,0.0,NaN,53708.0,Not healing,-108.641975,72
197471,NaN,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,...,9996014_1,NaN,9.0,2.0,0.0,NaN,53708.0,Grey,23.076923,75
197472,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,9996014_1,NaN,9.0,2.0,0.0,NaN,53708.0,Not healing,15.384615,80
197473,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,9996014_1,NaN,9.0,2.0,0.0,NaN,53708.0,Not healing,0.000000,87


## Check the datatype 

In [18]:
big_df.dtypes

Pulse                                    float64
AdiposeExposed                           float64
AdiposeNecrosisExposed                   float64
area                                     float64
AtrophieBlanche                          float64
BoneNecrosisExposed                      float64
BrawnyInduration                         float64
Callus                                   float64
CapsuleExposed                           float64
CapsuleNecrosisExposed                   float64
color                                    float64
CompCompression                          float64
CompDiet                                 float64
CompDressing                             float64
CompExercise                             float64
CompHBOVisits                            float64
CompLimb                                 float64
CompMedication                           float64
CompOffLoad                              float64
CompSmoking                              float64
CompWoundVisits     

In [19]:
#big_df = big_df.fillna(-99999)

## Change the datatype of the categorical values

In [20]:
categorical_features = [
    'color', 'CompCompression', 'CompDiet', 'CompDressing', 'CompExercise',
    'CompHBOVisits', 'CompLimb', 'CompMedication', 'CompOffLoad', 'CompSmoking',
    'CompWoundVisits', 'ExudateAmount', 'imageFileCount', 'lowerExtremityEdema',
    'NurseOnlyVisit', 'Odor', 'Pain', 'Stage', 'surroundingSkin',
    'surroundingSkinColor', 'SystematicInfection', 'WoundMargin', 'eschar',
    'AccidentType', 'acquiredAtFacility', 'ActiveWound', 'Anterior_Posterior',
    'Anterior_Dorsal', 'ClusteredWound', 'Inferior_Superior', 'Left_Right',
    'Location', 'PendingAmputationOnPresentation', 'Proximal_Distal',
    'ResolvedRecurrence', 'ResultOfAccident', 'Type', 'WoundCondition',
    'WoundRecurrence', 'woundstatus', 'WoundLocation', 'AdmissionStatusCode',
    'MedicareAdmission', 'PalliativeCare', 'CauseOfDeath', 'ethnicityCdcCode',
    'patientEthnicity', 'PatientGender', 'PatientSelfCare', 'Religion',
    'SmokingStatus', 'SmokingStatusText', 'DebridementLevel', 'Method',
    'Bleeding', 'IsThisBurn', 'SInsuranceZip', 'PInsuranceZip',
    'Wound_healing_status'
]



# Iterate over columns
for col in big_df.columns:
    for prefix in categorical_features:
        if col.startswith(prefix) or col == prefix:
            big_df[col] = big_df[col].astype('object')

# Verify the changes
print(big_df.dtypes)

Pulse                                    float64
AdiposeExposed                           float64
AdiposeNecrosisExposed                   float64
area                                     float64
AtrophieBlanche                          float64
BoneNecrosisExposed                      float64
BrawnyInduration                         float64
Callus                                   float64
CapsuleExposed                           float64
CapsuleNecrosisExposed                   float64
color                                     object
CompCompression                           object
CompDiet                                  object
CompDressing                              object
CompExercise                              object
CompHBOVisits                             object
CompLimb                                  object
CompMedication                            object
CompOffLoad                               object
CompSmoking                               object
CompWoundVisits     

## Label encoding the categorical variables

In [21]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Iterate over each column in the dataframe
for column in big_df.columns:
    # Check if the column is of object or categorical type
    if big_df[column].dtype == 'object':
        print(column)
        # Fit and transform the categorical column
        big_df[column] = label_encoder.fit_transform(big_df[column])

color
CompCompression
CompDiet
CompDressing
CompExercise
CompHBOVisits
CompLimb
CompMedication
CompOffLoad
CompSmoking
CompWoundVisits
ExudateAmount
imageFileCount
lowerExtremityEdema
NormalColor
NormalMoisture
normalTexture
NurseOnlyVisit
Odor
Pain
SSAbscess
SSCAndSPending
SSChangesInGranulationTissue
SSConfirmedLocal
SSConfirmedSystemic
SSDrainageDisproportionate
SSDrainageThick
SSFever
SSFoulWoundOdor
SSIncreasedEdema
SSInflammationExtendingToBone
SSMalaise
SSNewOnset
SSOther
SSPocketing
SSPoorlyDefinedErythemalBorder
SSSignsAndSymptoms
SSSystemicAntibiotics
SSTopicalAntibiotics
SSWarmthExtendingBeyondLocalPeriWound
SSWoundStalling
SSXRay
Stage
surroundingSkin
surroundingSkinColor
SystematicInfection
WoundMargin
eschar
DateAdded_woundassessment
Woundassessment_Title
Woundassessment_DocumentType
AccidentType
acquiredAtFacility
ActiveWound
Anterior_Posterior
Anterior_Dorsal
ClusteredWound
DateAcquired
DateInactive
DateOfAccident
DateWoundHealed
firstAssessmentDate
Inferior_Superior
Le

In [22]:
big_df.to_csv("big_df_1.csv")

In [61]:
big_df=pd.read_csv("big_df_1.csv")
big_df=big_df.drop(["Unnamed: 0"],axis=1)
big_df

,Pulse,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,...,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip,Wound_healing_status,Wound_healing_percentage,DayNumber
0,NaN,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,4,NaN,1
1,NaN,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,-13.095238,8
2,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,-6.000000,15
3,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,0.000000,22
4,NaN,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,0,25.521351,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,NaN,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,-108.641975,72
197471,NaN,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,0,23.076923,75
197472,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,15.384615,80
197473,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,0.000000,87


## See the important features according to RF, LightGBM and XGBoost

In [62]:
#from sklearn.ensemble import HistGradientBoostingClassifier

In [63]:
# target_column = 'Wound_healing_status'

# # Split the data into features (X) and the target variable (y)
# X = big_df.drop([target_column, 'DateAdded_woundassessment', 'Wound_healing_percentage'], axis=1)
# y = big_df[target_column]

# # Create a Random Forest classifier
# #rf = RandomForestClassifier()
# rf=HistGradientBoostingClassifier()

# # Fit the model
# rf.fit(X, y)

# # Get the feature importances
# feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# # Sort the feature importances in descending order
# feature_importances = feature_importances.sort_values(ascending=False)

# # Print the feature importances
# print(feature_importances)


In [64]:
# from sklearn.inspection import permutation_importance

# # Calculate permutation feature importances
# result = permutation_importance(rf, X, y, n_repeats=10, random_state=42)

# # Print feature importances
# for feature, importance in zip(X.columns, result.importances_mean):
#     print(f"{feature}: {importance:.4f}")

In [65]:
# feature_importances.index.tolist()

In [66]:
target_column = 'Wound_healing_status'

# Split the data into features (X) and the target variable (y)
X = big_df.drop([target_column, 'DateAdded_woundassessment', 'Wound_healing_percentage'], axis=1)
y = big_df[target_column]

# Create a Random Forest classifier
rf = LGBMClassifier()

# Fit the model
rf.fit(X, y)

# Get the feature importances
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# Sort the feature importances in descending order
feature_importances = feature_importances.sort_values(ascending=False)

# Print the feature importances
print(feature_importances)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021007 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9106
[LightGBM] [Info] Number of data points in the train set: 197475, number of used features: 169
[LightGBM] [Info] Start training from score -1.794422
[LightGBM] [Info] Start training from score -3.125859
[LightGBM] [Info] Start training from score -2.463936
[LightGBM] [Info] Start training from score -0.542952
[LightGBM] [Info] Start training from score -2.089514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
area                                     1138
DayNumber                                1091
Volume                                    782
Width                                     663
imageFileCount                            569
Woundassessment_VisitID                   489
Length                  

In [67]:
feature_importances.index.tolist()

['area',
 'DayNumber',
 'Volume',
 'Width',
 'imageFileCount',
 'Woundassessment_VisitID',
 'Length',
 'woundProgress',
 'Type',
 'Stage',
 'slough',
 'PatientZIP',
 'Location',
 'Epithialization',
 'Weight',
 'DateAdded_patient',
 'Age',
 'Depth',
 'Woundassessment_Title',
 'BMI',
 'RedGranulation',
 'CompCompression',
 'DateAdded_wound',
 'PatientIDWoundNumber',
 'tunneling',
 'AdmissionDate',
 'PatientDOB',
 'DateAcquired',
 'firstAssessmentDate',
 'Religion',
 'MedicareAdmission',
 'Admission_dateAdded',
 'PalliativeCare',
 'Height',
 'WoundMargin',
 'ExudateAmount',
 'Exudate',
 'RaceCodeList',
 'Pain',
 'Patient_Score',
 'NonWoundDiagnosis',
 'PatientGender',
 'WoundLocation',
 'DateInactive',
 'PInsuranceZip',
 'SmokingStatusText',
 'AdmissionStatusCode',
 'DateWoundHealed',
 'Method',
 'PatientSelfCare',
 'Odor',
 'SInsuranceZip',
 'color',
 'SloughLessThan20',
 'Notes',
 'ethnicityCdcCode',
 'DateOfDeath',
 'ActiveWound',
 'Pulse',
 'CompDiet',
 'Edema',
 'patientEthnicity',
 

In [68]:
target_column = 'Wound_healing_status'

# Split the data into features (X) and the target variable (y)
X = big_df.drop([target_column, 'DateAdded_woundassessment', 'Wound_healing_percentage'], axis=1)
y = big_df[target_column]

# Create a Random Forest classifier
rf = XGBClassifier()

# Fit the model
rf.fit(X, y)

# Get the feature importances
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)

# Sort the feature importances in descending order
feature_importances = feature_importances.sort_values(ascending=False)

# Print the feature importances
print(feature_importances)


DayNumber                                0.196877
area                                     0.110084
woundProgress                            0.052239
imageFileCount                           0.048929
CompCompression                          0.040504
Volume                                   0.038241
ActiveWound                              0.021993
Width                                    0.015711
TunnelingOrUndermining                   0.014820
tunneling                                0.013556
IsThisBurn                               0.013356
CompDiet                                 0.007131
Length                                   0.006988
Type                                     0.006512
redgran                                  0.006407
Callus                                   0.006318
Epithialization                          0.006005
PatientZIP                               0.005937
Anterior_Dorsal                          0.005744
Depth                                    0.005728


In [69]:
feature_importances.index.tolist()

['DayNumber',
 'area',
 'woundProgress',
 'imageFileCount',
 'CompCompression',
 'Volume',
 'ActiveWound',
 'Width',
 'TunnelingOrUndermining',
 'tunneling',
 'IsThisBurn',
 'CompDiet',
 'Length',
 'Type',
 'redgran',
 'Callus',
 'Epithialization',
 'PatientZIP',
 'Anterior_Dorsal',
 'Depth',
 'Exudate',
 'Edema',
 'Method',
 'woundstatus',
 'RedGranulation',
 'Excoriation',
 'slough',
 'Location',
 'MedicareAdmission',
 'SInsuranceZip',
 'patientEthnicity',
 'MuscleNecrosisExposed',
 'Friable',
 'DebridementLevel',
 'ResultOfAccident',
 'PatientSelfCare',
 'Stage',
 'ExudateAmount',
 'sinustract',
 'SloughLessThan20',
 'WoundLocation',
 'AdiposeNecrosisExposed',
 'color',
 'Denuded',
 'PatientDOB',
 'PalliativeCare',
 'PInsuranceZip',
 'PatientIDWoundNumber',
 'Woundassessment_VisitID',
 'AdmissionDate',
 'CauseOfDeath',
 'DateAdded_patient',
 'WoundMargin',
 'DateInactive',
 'Induration',
 'Rash',
 'Weight',
 'DateOfDeath',
 'SmokingStatus',
 'BMI',
 'surroundingSkin',
 'AdmissionSta

## Remove the less important features according to these models

In [70]:
# removal_DE_rf=['WoundRecurrence',
#  'GranulationQualityRed',
#  'ClusteredWound',
#  'Woundassessment_DocumentType',
#  'ErythemaLocCircumferential',
#  'PendingAmputationOnPresentation',
#  'NecroticTypeBlackEschar',
#  'NecroticTypeWetHydrated',
#  'NecroticTypeYellowSlough',
#  'TendernessOnPalpation',
#  'GranulationQualityHypertrophic',
#  'GranulationQualityPale',
#  'hypergran',
#  'GranulationQualityPink']    

removal_DE_lightgbm=['CompMedication',
 'CompExercise',
 'CompDressing',
 'CompHBOVisits',
 'Cyanosis',
 'ExposedBone',
 'CompLimb',
 'CompWoundVisits',
 'CompSmoking',
 'CompOffLoad',
 'Patient_DocumentType',
 'NurseOnlyVisit',
 'WoundRecurrence',
 'SSIncreasedEdema',
 'SSTopicalAntibiotics',
 'SSPoorlyDefinedErythemalBorder',
 'SSPocketing',
 'SSOther',
 'SSNewOnset',
 'SSMalaise',
 'SSInflammationExtendingToBone',
 'SSFoulWoundOdor',
 'PendingAmputationOnPresentation',
 'SSFever',
 'SSDrainageThick',
 'SSDrainageDisproportionate',
 'SSConfirmedSystemic',
 'SSChangesInGranulationTissue',
 'SSAbscess',
 'Pallor',
 'SSWoundStalling',
 'NonGranulationBiofilm',
 'ErythemaLocCircumferential',
 'NecroticTypeBlackEschar',
 'GranulatedHyperGranulated',
 'GranulationQualityPale',
 'GranulationQualityPink',
 'GranulationQualityRed',
 'ClusteredWound',
 'acquiredAtFacility',
 'hypergran',
 'Woundassessment_DocumentType',
 'lowerExtremityEdema',
 'NecroticTypeYellowSlough',
 'GranulationQualityHypertrophic',
 'DryEscharFluctance',
 'NecroticTypeWetHydrated',
 'Bogginess',
 'TendernessOnPalpation',
 'SSWarmthExtendingBeyondLocalPeriWound']

removal_DE_xgboost=['CompOffLoad',
 'CompSmoking',
 'ExposedBone',
 'Patient_DocumentType',
 'Cyanosis',
 'CompMedication',
 'CompDressing',
 'CompExercise',
 'CompHBOVisits',
 'CompLimb',
 'CompWoundVisits',
 'hypergran',
 'GranulatedHyperGranulated',
 'NonGranulationBiofilm',
 'SSWoundStalling',
 'SSPoorlyDefinedErythemalBorder',
 'SSPocketing',
 'SSOther',
 'SSNewOnset',
 'SSMalaise',
 'SSInflammationExtendingToBone',
 'SSIncreasedEdema',
 'SSFoulWoundOdor',
 'SSFever',
 'SSDrainageThick',
 'SSDrainageDisproportionate',
 'SSChangesInGranulationTissue',
 'SSAbscess',
 'NurseOnlyVisit',
 'NonGranulationDessicated',
 'ErythemaLocCircumferential',
 'WoundRecurrence',
 'NecroticTypeBlackEschar',
 'GranulationQualityPale',
 'GranulationQualityPink',
 'PendingAmputationOnPresentation',
 'GranulationQualityRed',
 'ClusteredWound',
 'Pallor',
 'acquiredAtFacility',
 'lowerExtremityEdema',
 'Woundassessment_DocumentType',
 'NecroticTypeYellowSlough',
 'GranulationQualityHypertrophic',
 'DryEscharFluctance',
 'NecroticTypeWetHydrated',
 'Bogginess',
 'TendernessOnPalpation',
 'SSWarmthExtendingBeyondLocalPeriWound']


# removal_DE_rf=["GranulationQualityHypertrophic", "NecroticTypeYellowSlough", "TendernessOnPalpation", "NecroticTypeBlackEschar",
#                "ErythemaLocCircumferential", "PendingAmputationOnPresentation", "ClusteredWound", "WoundRecurrence",
#                "hypergran","GranulationQualityRed", "GranulationQualityPink", "NecroticTypeWetHydrated", 
#                "GranulationQualityPale"]    

# removal_DE_lightgbm=["SSChangesInGranulationTissue", "CompHBOVisits", "SSPoorlyDefinedErythemalBorder", "SSPocketing", 
#                      "CompWoundVisits", "SSOther", "SSNewOnset", "CompSmoking", "CompOffLoad", "CompMedication", "CompLimb", 
#                      "CompExercise", "SSWoundStalling", "CompDressing", "SSMalaise", "SSInflammationExtendingToBone", 
#                      "SSIncreasedEdema","SSFoulWoundOdor", "SSFever", "BoneNecrosisExposed", "SSDrainageThick", 
#                      "SSDrainageDisproportionate","Diabetes","Cyanosis","SSAbscess","lowerExtremityEdema","Pallor",
#                      "PendingAmputationOnPresentation","NonGranulationBiofilm","ClusteredWound","NonGranulationDessicated",
#                      "acquiredAtFacility","hypergran","WoundRecurrence","NurseOnlyVisit","NecroticTypeYellowSlough",
#                      "GranulationQualityRed","GranulationQualityPink","GranulationQualityPale","GranulatedHyperGranulated",
#                      "GranulationQualityHypertrophic","DryEscharFluctance","NecroticTypeWetHydrated","Bogginess",
#                      "TendernessOnPalpation","ExposedBone","NecroticTypeBlackEschar","ErythemaLocCircumferential",
#                      "SSWarmthExtendingBeyondLocalPeriWound"]

# removal_DE_xgboost=["hypergran","GranulationQualityPink","GranulationQualityRed","Diabetes","NonGranulationBiofilm",
#                     "lowerExtremityEdema","Patient_DocumentType","GranulationQualityPale","GranulatedHyperGranulated",
#                     "CompExercise","CapsuleNecrosisExposed","CompDressing","NonGranulationDessicated","CompSmoking",
#                     "CompOffLoad","CompMedication","CompLimb","CompHBOVisits","CompWoundVisits","Bogginess","NurseOnlyVisit",
#                     "Pallor","NecroticTypeWetHydrated","DryEscharFluctance","GranulationQualityHypertrophic",
#                     "NecroticTypeYellowSlough","NecroticTypeBlackEschar","ErythemaLocCircumferential","acquiredAtFacility",
#                     "ClusteredWound","SSWoundStalling","SSPoorlyDefinedErythemalBorder","SSPocketing","SSOther","SSNewOnset",
#                     "SSMalaise","SSInflammationExtendingToBone","PendingAmputationOnPresentation","SSIncreasedEdema",
#                     "SSFoulWoundOdor","SSFever","WoundRecurrence","SSDrainageThick","SSDrainageDisproportionate",
#                     "SSChangesInGranulationTissue","TendernessOnPalpation","SSAbscess","SSWarmthExtendingBeyondLocalPeriWound"]

In [71]:
#common_elements = set(removal_DE_rf) & set(removal_DE_lightgbm) & set(removal_DE_xgboost)
common_elements = set(removal_DE_lightgbm) & set(removal_DE_xgboost)


len(common_elements)

48

In [72]:
all_elements = set(removal_DE_lightgbm) | set(removal_DE_xgboost)

len(all_elements)

51

In [73]:
list(all_elements)

['TendernessOnPalpation',
 'Cyanosis',
 'Bogginess',
 'CompOffLoad',
 'CompHBOVisits',
 'SSPocketing',
 'SSNewOnset',
 'SSMalaise',
 'CompSmoking',
 'PendingAmputationOnPresentation',
 'Pallor',
 'NonGranulationDessicated',
 'GranulatedHyperGranulated',
 'SSDrainageThick',
 'acquiredAtFacility',
 'NonGranulationBiofilm',
 'NurseOnlyVisit',
 'SSTopicalAntibiotics',
 'SSIncreasedEdema',
 'GranulationQualityPink',
 'NecroticTypeBlackEschar',
 'SSWoundStalling',
 'hypergran',
 'SSWarmthExtendingBeyondLocalPeriWound',
 'SSInflammationExtendingToBone',
 'SSFever',
 'CompMedication',
 'ClusteredWound',
 'SSOther',
 'GranulationQualityPale',
 'SSFoulWoundOdor',
 'Woundassessment_DocumentType',
 'SSConfirmedSystemic',
 'CompExercise',
 'NecroticTypeYellowSlough',
 'CompLimb',
 'SSAbscess',
 'Patient_DocumentType',
 'ErythemaLocCircumferential',
 'WoundRecurrence',
 'SSDrainageDisproportionate',
 'GranulationQualityRed',
 'SSChangesInGranulationTissue',
 'ExposedBone',
 'NecroticTypeWetHydrated'

## So let's remove these data elements which less important according to feature importance for rf, lightgbm and xgboost

In [74]:
big_df_1=big_df.drop(list(all_elements), axis=1)
big_df_1

,Pulse,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,...,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip,Wound_healing_status,Wound_healing_percentage,DayNumber
0,NaN,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,4,NaN,1
1,NaN,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,-13.095238,8
2,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,-6.000000,15
3,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,0.000000,22
4,NaN,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,0,25.521351,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,NaN,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,-108.641975,72
197471,NaN,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,0,23.076923,75
197472,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,15.384615,80
197473,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,0.000000,87


In [75]:
# 3 = not healing
# 0 = grey
# 2 = healing
# 4 = unknown
# 1 = healed

## Removing highly correlated features from the dataframe

In [76]:
big_df_1=big_df_1.drop(["DateAdded_woundassessment"],axis=1)

In [77]:
# target_column='Wound_healing_status'

# # Set the threshold for correlation
# threshold = 0.90
# #0.8 should be thres

# #wilconson rank_sum correlation
# #tau as coefficient

# # List to store reduced DataFrames
# dfs_reduced = []


df=big_df_1

# # Calculate correlation matrix
# #corr_matrix = df.drop(target_column, axis=1).corr()
# correlation_matrix = df.corr()

# # Step 2: Identify Highly Correlated Features
# threshold = 0.95
# highly_correlated_pairs = []

# for i in range(len(df.columns)):
#     for j in range(i + 1, len(df.columns)):
#         if abs(correlation_matrix.iloc[i, j]) > threshold:
#             highly_correlated_pairs.append((df.columns[i], df.columns[j]))

# # Step 3 and 4: Evaluate Feature Importance and Drop Less Important Features
# for feature1, feature2 in highly_correlated_pairs:
#     # Create a subset DataFrame with the correlated features and the target feature
#     if all(feature in df.columns for feature in [feature1, feature2]):
#         subset_df = pd.concat([df[[feature1, feature2]],df[target_column]],axis=1)

#         # Prepare data for Random Forest
#         X = subset_df[[feature1, feature2]]
#         y = subset_df[target_column]

#         # Fit Random Forest regressor
#         rf = RandomForestRegressor()
#         rf.fit(X, y)

#         # Print feature importance
#         print(f"\nFeature Importance for {feature1}: {rf.feature_importances_[0]}")
#         print(f"Feature Importance for {feature2}: {rf.feature_importances_[1]}")

#         # Drop the less important feature
#         if rf.feature_importances_[0] < rf.feature_importances_[1]:
#             df.drop(feature1, axis=1, inplace=True)
#             print("Dropping",feature1,"\n")
#         else:
#             df.drop(feature2, axis=1, inplace=True)
#             print("Dropping",feature2,"\n")

# # Display the modified DataFrame
# display(df)

# #     # Append the reduced DataFrame to the list
# #     dfs_reduced.append(df)

# # # Concatenate the reduced DataFrames
# # df_reduced = pd.concat(dfs_reduced, axis=1)

# # # Display the final reduced DataFrame
# # df_reduced


# #scaled log transform


In [78]:
df_filtered=df
df_filtered

,Pulse,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,...,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip,Wound_healing_status,Wound_healing_percentage,DayNumber
0,NaN,-1.0,1.0,33.60,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,4,NaN,1
1,NaN,-1.0,1.0,38.00,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,-13.095238,8
2,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,-6.000000,15
3,NaN,-1.0,1.0,40.28,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,3,0.000000,22
4,NaN,-1.0,1.0,30.00,1.0,1.0,1.0,1.0,1.0,1.0,...,0,0,13,1,1,68,64,0,25.521351,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,NaN,1.0,1.0,169.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,-108.641975,72
197471,NaN,1.0,1.0,130.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,0,23.076923,75
197472,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,15.384615,80
197473,NaN,1.0,1.0,110.00,1.0,1.0,1.0,1.0,1.0,1.0,...,24436,3,6,1,1,68,43,3,0.000000,87


In [80]:
# Example dataset (replace this with your actual dataset)
data = df

# Custom correlation function to handle NaN values
def custom_correlation(x, y, method):
    valid_values = ~np.isnan(x) & ~np.isnan(y)
    x_valid = x[valid_values]
    y_valid = y[valid_values]
    
    if method == 'pearson':
        # Handle NaN values for Pearson correlation
        mask = np.isfinite(x_valid) & np.isfinite(y_valid)
        correlation_result = pointbiserialr(x_valid[mask], y_valid[mask])
    elif method == 'pointbiserial':
        correlation_result = pointbiserialr(x_valid, y_valid)
    else:
        raise ValueError(f"Unsupported correlation method: {method}")
    
    return correlation_result

# Create an empty dictionary to store correlation results
correlation_results = {}

# Iterate through numerical columns
for numerical_var in data.select_dtypes(include=['float64', 'int64']).columns:
    # Iterate through all columns
    for column in data.columns:
        # Skip the numerical variable itself
        if column == numerical_var:
            continue
        
        # Determine data types
        data_type_numerical = data[numerical_var].dtype
        data_type_column = data[column].dtype
        
        # Apply the appropriate correlation method based on data types
        if data_type_numerical in ['float64', 'int64'] and data_type_column in ['float64', 'int64']:
            # Both variables are numerical - use Pearson correlation
            correlation_result = custom_correlation(data[numerical_var], data[column], 'pearson')
        elif data_type_numerical in ['float64', 'int64'] and data_type_column == 'object':
            # Numerical variable and categorical variable - use point-biserial correlation
            # Convert binary categorical data to numeric
            data_numeric = pd.Categorical(data[column]).codes
            correlation_result = custom_correlation(data[numerical_var], data_numeric, 'pointbiserial')
        # Add other cases as needed
        else:
            # Other combinations - skip or handle accordingly
            continue
        
        # Store the correlation result
        key = f"{numerical_var}_vs_{column}"
        correlation_results[key] = correlation_result

# Print the results
for key, result in correlation_results.items():
    print(f"Correlation {key}:\n{result}\n")


/home/jupyter-apaddo/.local/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Correlation Pulse_vs_AdiposeExposed:
SignificanceResult(statistic=0.160550766459909, pvalue=9.657573449368926e-233)

Correlation Pulse_vs_AdiposeNecrosisExposed:
SignificanceResult(statistic=0.15706246163420381, pvalue=1.1071951114152581e-222)

Correlation Pulse_vs_area:
SignificanceResult(statistic=0.018270543494789068, pvalue=0.00023041251113448385)

Correlation Pulse_vs_AtrophieBlanche:
SignificanceResult(statistic=-0.034207002656508945, pvalue=5.306481053909153e-12)

Correlation Pulse_vs_BoneNecrosisExposed:
SignificanceResult(statistic=0.19468068086584528, pvalue=0.0)

Correlation Pulse_vs_BrawnyInduration:
SignificanceResult(statistic=-0.01704275383825709, pvalue=0.0005913329801453637)

Correlation Pulse_vs_Callus:
SignificanceResult(statistic=-0.12055935025189145, pvalue=2.174906453727132e-131)

Correlation Pulse_vs_CapsuleExposed:
SignificanceResult(statistic=0.19669778784496506, pvalue=0.0)

Correlation Pulse_vs_CapsuleNecrosisExposed:
SignificanceResult(statistic=0.2015106948

In [81]:
X = df.drop([target_column, 'Wound_healing_percentage'], axis=1)
y = df[target_column]

# Fit Random Forest regressor
rf = LGBMClassifier()
rf.fit(X, y)

for key, result in correlation_results.items():
    #print(key)
    # Split the string into two parts
    split_parts = key.split('_vs_')
    # Display the result
    
    
    if round(result[0],4)>0.8:
        
        print(split_parts)
        print("Correlation Coeff:",round(result[0],4))
        feature1=split_parts[0]
        feature2=split_parts[1]

        columns_to_check = [feature1, feature2]

        # Iterate through the columns and check if each column is present
        if set(columns_to_check).issubset(df.columns):
            print("Both columns are present in the DataFrame.")

            # Print feature importance
            print(f"\nFeature Importance for {feature1}: {pd.Series(rf.feature_importances_, index=X.columns)[feature1]}")
            print(f"Feature Importance for {feature2}: {pd.Series(rf.feature_importances_, index=X.columns)[feature2]}")

            feature_1_importance=pd.Series(rf.feature_importances_, index=X.columns)[feature1]
            feature_2_importance=pd.Series(rf.feature_importances_, index=X.columns)[feature2]

            # Drop the less important feature
            if feature_1_importance < feature_2_importance:
                df.drop(feature1, axis=1, inplace=True)
                print("Dropping",feature1,"\n")
            else:
                df.drop(feature2, axis=1, inplace=True)
                print("Dropping",feature2,"\n")

    # Display the modified DataFrame
display(df)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018612 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8996
[LightGBM] [Info] Number of data points in the train set: 197475, number of used features: 132
[LightGBM] [Info] Start training from score -1.794422
[LightGBM] [Info] Start training from score -3.125859
[LightGBM] [Info] Start training from score -2.463936
[LightGBM] [Info] Start training from score -0.542952
[LightGBM] [Info] Start training from score -2.089514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
['Pulse', 'Epithialization']
Correlation Coeff: 0.8344
Both columns are present in the DataFrame.

Feature Importance for Pulse: 57
Feature Importance for Epithialization: 257
Dropping Pulse 

['Pulse', 'RedGranulation']
Correlation Coeff: 0.8357
['Pulse', 'slough']
Correlation Coeff: 0.84
['Adipose

,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BrawnyInduration,Callus,color,CompCompression,CompDiet,Crepitus,...,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip,Wound_healing_status,Wound_healing_percentage,DayNumber
0,-1.0,1.0,33.60,1.0,1.0,1.0,13,1,0,1.0,...,0,0,13,1,1,68,64,4,NaN,1
1,-1.0,1.0,38.00,1.0,1.0,1.0,13,1,0,1.0,...,0,0,13,1,1,68,64,3,-13.095238,8
2,-1.0,1.0,40.28,1.0,1.0,1.0,2,1,0,1.0,...,0,0,13,1,1,68,64,3,-6.000000,15
3,-1.0,1.0,40.28,1.0,1.0,1.0,2,1,0,1.0,...,0,0,13,1,1,68,64,3,0.000000,22
4,-1.0,1.0,30.00,1.0,1.0,1.0,2,1,0,1.0,...,0,0,13,1,1,68,64,0,25.521351,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197470,1.0,1.0,169.00,1.0,1.0,1.0,8,0,0,1.0,...,24436,3,6,1,1,68,43,3,-108.641975,72
197471,1.0,1.0,130.00,1.0,1.0,1.0,8,0,0,1.0,...,24436,3,6,1,1,68,43,0,23.076923,75
197472,1.0,1.0,110.00,1.0,1.0,1.0,8,0,0,1.0,...,24436,3,6,1,1,68,43,3,15.384615,80
197473,1.0,1.0,110.00,1.0,1.0,1.0,8,0,0,1.0,...,24436,3,6,1,1,68,43,3,0.000000,87


In [82]:
# target_column='Wound_healing_status'


# df=big_df_1

# # Calculate correlation matrix
# #corr_matrix = df.drop(target_column, axis=1).corr()
# correlation_matrix = df.corr()

# # Step 2: Identify Highly Correlated Features
# threshold = 0.95
# highly_correlated_pairs = []

# for i in range(len(df.columns)):
#     for j in range(i + 1, len(df.columns)):
#         if abs(correlation_matrix.iloc[i, j]) > threshold:
#             highly_correlated_pairs.append((df.columns[i], df.columns[j]))
            
# # Prepare data for Random Forest
# X = df.drop([target_column, 'Wound_healing_percentage'], axis=1)
# y = df[target_column]

# # Fit Random Forest regressor
# rf = LGBMClassifier()
# rf.fit(X, y)

# # Step 3 and 4: Evaluate Feature Importance and Drop Less Important Features
# for feature1, feature2 in highly_correlated_pairs:
#     # List of columns you want to check
#     columns_to_check = [feature1, feature2]

#     # Iterate through the columns and check if each column is present
#     if set(columns_to_check).issubset(df.columns):
#         print("Both columns are present in the DataFrame.")
        
#         # Print feature importance
#         print(f"\nFeature Importance for {feature1}: {pd.Series(rf.feature_importances_, index=X.columns)[feature1]}")
#         print(f"Feature Importance for {feature2}: {pd.Series(rf.feature_importances_, index=X.columns)[feature2]}")

#         feature_1_importance=pd.Series(rf.feature_importances_, index=X.columns)[feature1]
#         feature_2_importance=pd.Series(rf.feature_importances_, index=X.columns)[feature2]

#         # Drop the less important feature
#         if feature_1_importance < feature_2_importance:
#             df.drop(feature1, axis=1, inplace=True)
#             print("Dropping",feature1,"\n")
#         else:
#             df.drop(feature2, axis=1, inplace=True)
#             print("Dropping",feature2,"\n")

# # Display the modified DataFrame
# display(df)

# #     # Append the reduced DataFrame to the list
# #     dfs_reduced.append(df)

# # # Concatenate the reduced DataFrames
# # df_reduced = pd.concat(dfs_reduced, axis=1)

# # # Display the final reduced DataFrame
# # df_reduced


In [83]:
# target_column='Wound_healing_status'


# df=big_df_1

# # Calculate correlation matrix
# #corr_matrix = df.drop(target_column, axis=1).corr()
# correlation_matrix = df.corr()

# # Step 2: Identify Highly Correlated Features
# threshold = 0.95
# highly_correlated_pairs = []

# for i in range(len(df.columns)):
#     for j in range(i + 1, len(df.columns)):
#         if abs(correlation_matrix.iloc[i, j]) > threshold:
#             highly_correlated_pairs.append((df.columns[i], df.columns[j]))

# # Step 3 and 4: Evaluate Feature Importance and Drop Less Important Features
# for feature1, feature2 in highly_correlated_pairs:
#     # Create a subset DataFrame with the correlated features and the target feature
#     if all(feature in df.columns for feature in [feature1, feature2]):
#         subset_df = pd.concat([df[[feature1, feature2]],df[target_column]],axis=1)

#         # Prepare data for Random Forest
#         X = subset_df[[feature1, feature2]]
#         y = subset_df[target_column]

#         # Fit Random Forest regressor
#         rf = XGBClassifier()
#         rf.fit(X, y)

#         # Print feature importance
#         print(f"\nFeature Importance for {feature1}: {rf.feature_importances_[0]}")
#         print(f"Feature Importance for {feature2}: {rf.feature_importances_[1]}")

#         # Drop the less important feature
#         if rf.feature_importances_[0] < rf.feature_importances_[1]:
#             df.drop(feature1, axis=1, inplace=True)
#             print("Dropping",feature1,"\n")
#         else:
#             df.drop(feature2, axis=1, inplace=True)
#             print("Dropping",feature2,"\n")

# # Display the modified DataFrame
# display(df)

# #     # Append the reduced DataFrame to the list
# #     dfs_reduced.append(df)

# # # Concatenate the reduced DataFrames
# # df_reduced = pd.concat(dfs_reduced, axis=1)

# # # Display the final reduced DataFrame
# # df_reduced


In [84]:
df["Wound_healing_status"].unique()

array([4, 3, 0, 2, 1])

In [85]:
df.columns.tolist()

['AdiposeExposed',
 'AdiposeNecrosisExposed',
 'area',
 'AtrophieBlanche',
 'BrawnyInduration',
 'Callus',
 'color',
 'CompCompression',
 'CompDiet',
 'Crepitus',
 'Denuded',
 'Depth',
 'DryScaly',
 'Ecchymosis',
 'Edema',
 'Erythema',
 'Excoriation',
 'ExposedMuscle',
 'ExposedTendon',
 'Exudate',
 'ExudateAmount',
 'Fluctuance',
 'Friable',
 'greygran',
 'Hemosiderosis',
 'imageFileCount',
 'Induration',
 'Length',
 'Maceration',
 'Moist',
 'NoMeasurableDepth',
 'NormalColor',
 'NormalMoisture',
 'normalTexture',
 'Odor',
 'Pain',
 'pinkgran',
 'Rash',
 'redgran',
 'Rubor',
 'Scarring',
 'Shiny',
 'sinustract',
 'slough',
 'SloughLessThan20',
 'SSCAndSPending',
 'SSConfirmedLocal',
 'SSSignsAndSymptoms',
 'SSSystemicAntibiotics',
 'SSXRay',
 'Stage',
 'surroundingSkin',
 'surroundingSkinColor',
 'SystematicInfection',
 'Temperature',
 'TendonNecrosisExposed',
 'tunneling',
 'TunnelingOrUndermining',
 'Width',
 'woundProgress',
 'GranulationTexture',
 'WoundMargin',
 'eschar',
 'Wound

In [86]:
df.to_csv("df_filtered.csv")